# 새 섹션

In [5]:
from google.colab import files
uploaded = files.upload()

Saving 서울시 주요 공원현황2.csv to 서울시 주요 공원현황2.csv


In [242]:
import pandas as pd
df = pd.read_csv('서울시 주요 공원현황.csv',encoding='EUC-KR')
df.head(10)

,연번,공원명,공원개요,면적,개원일,주요시설,주요식물,안내도,오시는길,이용시참고사항,이미지,지역,공원주소,관리부서,전화번호,X좌표(GRS80TM),Y좌표(GRS80TM),X좌표(WGS84),Y좌표(WGS84),바로가기
0,1,남산도시자연공원,남산공원은 도심에 위치하여 서울시민에게 맑은 공기를 제공하는 자연휴식처이며 산책 꽃...,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,1968.9.10,기반시설 : 광장 45950㎡ 도로 108530㎡ 산책로 6.7㎞ (북측:3.7㎞/...,소나무 단풍 아카시아 상수리나무 등 191종 2881870주,http://parks.seoul.go.kr/upload/seditorMulti/2...,명동역(남산케이블카 와룡묘 서울애니메이션센터 방면) 도보 명동역 3번출구 → 퍼시픽...,남산공원 차량통행 금지안내 2005년 5월 1일부터 남산공원에 일반승용차 택시 통행...,http://parks.seoul.go.kr/file/info/view.do?fId...,중구,서울특별시 중구 삼일대로 231(예장동),서울특별시중부푸른도시사업소,02-3783-5900,198364.107,450395.554,126.990377,37.550140,http://parks.seoul.go.kr/template/sub/namsan.do
1,2,길동생태공원,길동생태공원은 생물의 서식처를 제공하고 종다양성을 증진시키며 자연생태계의 생물들을 ...,80683㎡,1999.5.20,탐방객안내소 야외전시대 관찰대 목재데크 조류관찰대 길동생태문화센터,소나무 보리수 등 64종 31800주 산국 부들 등 138종 192800본,http://parks.seoul.go.kr/template/common/img/p...,NaN,매주 월요일은 휴관입니다 사전예약 후 입장하실 수 있습니다. ▶ 인터넷 예약 바로가...,http://parks.seoul.go.kr/file/info/view.do?fId...,강동구,서울특별시 강동구 천호대로 1291(길동생태공원),동부공원녹지사업소 길동생태공원,02-489-2770,213554.120,448852.675,127.154779,37.540394,http://parks.seoul.go.kr/template/sub/gildong.do
2,3,서울대공원,서울대공원은 세계 각국의 야생동물들이 살아 숨 쉬는 서울동물원과 다양한 재미와 즐거...,9132690m²,1984.5.1,동물원 식물원 테마가든(장미원 어린이동물원 피크닉장) 치유숲 산림욕장 캠핑장 국립현...,NaN,NaN,지하철 4호선 대공원역 하차(2번 출구) 도보 15분 정도 문의처 02)500-73...,일반사항 공원을 깨끗하게 이용합니다. 대중교통을 이용해 주세요. 기념물 시설물 풀과...,http://parks.seoul.go.kr/file/info/view.do?fId...,과천시,경기도 과천시 대공원광장로 102,전략기획실,02-500-7032,200994.267,437163.981,127.019847,37.426449,http://grandpark.seoul.go.kr/
3,4,서울숲,당초 골프장 승마장 등이 있던 뚝섬일대를 주거업무 지역으로 개발할 경우 약 4조원에...,480994㎡,2005.6.18,4개 테마공원: 문화예술공원 자연생태숲 자연체험학습원 습지생태원 주요시설: 가족마당...,수 목 : 소나무 섬잣나무 계수나무 외 95종 415795주 식물원 : 선인장 등 ...,http://parks.seoul.go.kr/template/common/img/p...,NaN,지하철 분당선 서울숲역 3번출구 도보로 약5분 2호선 뚝섬역 8번출구 도보로 약 1...,http://parks.seoul.go.kr/file/info/view.do?fId...,성동구,서울특별시 성동구 뚝섬로 273 (성수동1가),서울숲 관리사무소,02-460-2905,203695.432,449290.726,127.041798,37.543072,http://parks.seoul.go.kr/template/sub/seoulfor...
4,5,월드컵공원,월드컵공원은 서울의 서쪽에 위치하여 1978년부터 1993년까지 15년간 서울시민이...,2284085㎡,2002.5.1,평화의 공원 월드컵공원전시관(879㎡) 유니세프광장(2400㎡) 평화광장(5217㎡...,NaN,http://parks.seoul.go.kr/template/common/img/p...,지하철 6호선 월드컵경기장역 하차 → 1번 출구로 나온 후 직진 → 큰길(도로)이 ...,일반사항 공원을 깨끗하게 이용합니다. 대중교통을 이용해 주세요. 기념물 시설물 풀과...,http://parks.seoul.go.kr/file/info/view.do?fId...,마포구,서울특별시 마포구 하늘공원로 84(월드컵공원),월드컵공원,02-300-5500,190658.070,451598.831,126.878907,37.571805,http://parks.seoul.go.kr/template/sub/worldcup...
5,6,광화문시민열린마당,광화문 옛 육조거리가 있던 자리에 새롭게 탄생한 서울시민을 위한 쉼터 '시민열린마당...,8948.1㎡,1999.1.1,조 경 시 설 : 육조상징벽 6개소 전통담장 4개소 열주 22조 화계6개소 편 익 ...,수 목 : 소나무 느티나무 배롱나무등 27종 5535주 초 화 : 맥문동 꽃창포 수...,http://parks.seoul.go.kr/cheditor/attach/20110...,지하철 3호선 경복궁역 5번출구 5호선 광화문역 2번출구 버스(G 지선 B 간선 R...,일반사항 공원을 깨끗하게 이용합니다. 대중교통을 이용해 주세요. 기념물 시설물 풀과...,http://parks.seoul.go.kr/file/info/view.do?fId...,종로구,서울특별시 종로구 세종로 76-2,종로구청 공원녹지과,02-2148-2843,198054.179,452832.049,126.978023,37.575045,NaN
6,7,진관근린공원(구파발폭포),1993. 09. 06 서울시장 방침(제1400호) 통일로변 인공폭포조성계획 19...,휴양 및 편익시설 : 평의자 7개 화장실 23.18㎡ 조경시설 - 은하폭포(높이 1...,1997.10.1,폭포 야외무대,수 목 : 소나무 잣나무 주목 느티나무 은행나무 등 29종 6515주 초 화 : 구...,NaN,지하철 3호선 구파발역 버스(G 지선 B 간선 R 광역 Y 순환) 구파발역 또는 ...,일반사항 공원을 깨끗하게 이용합니다. 대중교통을 이용해 주세요. 기념물 시설물 풀과...,http://parks.seoul.go.kr/file/info/view.do?fId...,은평구,서울특별시 은평구 진관동 393,은평구청 공원녹지과,02-351-8004,192864.424,459694.746,126.919216,37.637024,NaN
7,8,중마루근린공원,서울시가 ‘공원녹지 확충 5개년 계획’ 에 의해 구 영등포시립병원(6동)을 이전한 ...,6456㎥,1997.11.25,기반시설 : 광장 200㎡ 산책 등산로 0.50㎞ 편익시설 : 광장 의자 휴지통 ...,수 목 : 소나무 느티나무 외 43종 3580주 초 화 : 비비추 옥잠화외 25종 ...,http://parks.seoul.go.kr/template/common/img/p...,지하철 1호선 영등포역 (도보15분소요) 5호선 영등포시장역3번 출구 경원극장까지 ...,일반사항 공원을 깨끗하게 이용합니다. 대중교통을 이용해 주세요. 기념물 시설물 풀과...,http://parks.seoul.go.kr/file/info/view.do?fId...,영등포구,서울특별시 영등포구 영등포동2가 222,영등포구청 푸른도시과,02-2670-3717,192162.090,446711.963,126.911267,37.519832,NaN
8,9,응봉공원,응봉은 산모양이 매처럼 보이기 때문에 붙여졌다고 하며 조선시대 왕들이 매사냥을 하여...,80309㎡ 배수지 시설용량 : 201.000㎥ 송수 펌프용량 : 직송급수계통 30...,2003.2.21,조경시설 : 파고라 외 222소

In [97]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

In [197]:
import re
number = re.compile('[^0-9]+')

In [243]:
m_list = []
for i in df.index:
  if type(df.면적[i]) == float:
    x = float('0')
  else:
    x = number.sub('',df.면적[i].split(' ')[0]).strip()
  m_list.append(x)
df['면적'] = m_list


In [244]:
for i in df.index:
  if df.면적[i] == '':
    df.면적[i] = '0'
  df.면적[i] = float(df.면적[i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [245]:
size_list,y_list = [],[]
for i in df.index:
  if float(df.면적[i]) <100000:
    size,y = '소형', '3'
  elif  float(df.면적[i]) <1000000:
    size,y = '중형', '7'
  else:
    size,y = '대형', '15'
  size_list.append(size)
  y_list.append(y)
df['분류'] = size_list
df['크기'] = y_list

In [246]:
pf = df[['공원명','지역','면적','X좌표(WGS84)','Y좌표(WGS84)','분류','크기']]
pf.columns = ['공원명','지역','면적','경도','위도','분류','크기']
pf

,공원명,지역,면적,경도,위도,분류,크기
0,남산도시자연공원,중구,2.89689e+06,126.990377,37.550140,대형,15
1,길동생태공원,강동구,80683,127.154779,37.540394,소형,3
2,서울대공원,과천시,9.13269e+06,127.019847,37.426449,대형,15
3,서울숲,성동구,480994,127.041798,37.543072,중형,7
4,월드컵공원,마포구,2.28408e+06,126.878907,37.571805,대형,15
...,...,...,...,...,...,...,...
127,용마도시자연공원(사가정공원),중랑구,109635,127.095997,37.579762,중형,7
128,문화비축기지,마포구,140022,126.893246,37.571718,중형,7
129,경춘선숲길,NaN,211392,127.076482,37.627077,중형,7
130,율현공원,강남구,157535,127.115594,37.472332,중형,7


In [247]:
pf = pf.dropna()
pf

,공원명,지역,면적,경도,위도,분류,크기
0,남산도시자연공원,중구,2.89689e+06,126.990377,37.550140,대형,15
1,길동생태공원,강동구,80683,127.154779,37.540394,소형,3
2,서울대공원,과천시,9.13269e+06,127.019847,37.426449,대형,15
3,서울숲,성동구,480994,127.041798,37.543072,중형,7
4,월드컵공원,마포구,2.28408e+06,126.878907,37.571805,대형,15
...,...,...,...,...,...,...,...
125,숭인근린공원,종로구,45615,127.017827,37.576442,소형,3
126,서일대뒷산공원,중랑구,631021,127.098782,37.587716,중형,7
127,용마도시자연공원(사가정공원),중랑구,109635,127.095997,37.579762,중형,7
128,문화비축기지,마포구,140022,126.893246,37.571718,중형,7


In [248]:
pf.to_csv('서울공원요약.csv', index=False)

In [239]:
import folium

In [249]:
map = folium.Map(location=(37.5,126.982), zoom_start=10.5)
for i in pf.index:
  folium.CircleMarker(
      radius=int(pf.크기[i]),            # int64는 처리 불가
      location=[pf.위도[i],pf.경도[i]],
      popup=folium.Popup(pf.공원명[i], max_width=200),
      tooltip=pf.공원명[i],
      color='crimson'
  ).add_to(map)
map

In [257]:
map = folium.Map(location=[37.5502, 126.982], zoom_start=11, tiles='Stamen Toner')
for i in pf.index:
    folium.CircleMarker([pf.위도[i], pf.경도[i]], 
                        radius=int(pf.크기[i]),
                        tooltip=f"{pf.공원명[i]}({pf.면적[i]}㎡)",
                        color='green', fill_color='green').add_to(map)
title_html = '<h3 align="center" style="font-size:20px"><b>서울시내 공원 위치와 크기</b></h3>'   
map.get_root().html.add_child(folium.Element(title_html))
map